# 파생변수 생성

이해원

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
pd.set_option('display.max_columns', None)

In [54]:
cont = pd.read_csv('SDB2_I_CT_CONT_PVT.csv')
rlps = pd.read_csv('rlps_fin3.csv')

In [59]:
rlps.head()

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,rltn_cd_x
0,8,795034,I,1,0
1,8,3700186,I,1,0
2,8,6394418,I,99,2
3,8,6614027,I,1,0
4,8,7083655,I,1,0


CT_CNCLS_DT 보험계약체결월 NVARCHAR(6)

CT_TRMNT_DT 보험계약해지월 NVARCHAR(6)

CT_ST_DT 보험시기월 NVARCHAR(6)

CT_END_DT 보험종기월 NVARCHAR(6)

CT_EFCTV_DT 보험실효월 NVARCHAR(6)

CT_RST_DT 보험부활월 NVARCHAR(6)

CT_PY_AMT 납입보험료 BIGINT

CT_PY_CYCLE_CD 납입주기코드 NVARCHAR(2)

CT_PY_PD 납입기간 BIGINT

# 0. 전처리

## (1) 특정 시점에 / 한 증권번호의 / 계약 건이 2개 이상 drop

In [55]:
pre_0 = cont.groupby('POL_SN')[['YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006']].sum().reset_index()

#이상치만 모으기
pre = pre_0[pre_0['YM_202006']>1] #13882개의 증권번호

#이상치 제거
cont = cont[~cont['POL_SN'].isin(pre['POL_SN'])].reset_index(drop=True)
cont

,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,0,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,0,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,0,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,0,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,0,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303237,8,2330274,1,4,5,201905,999912,201905,202005,999912,999912,59,3,1,0,1,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
303238,8,6242384,0,7,2,201212,999912,201212,203812,999912,999912,47,2,10,0,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303239,6,6050915,0,7,7,200902,201107,200902,201902,999912,999912,15,2,10,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303240,6,9901184,0,14,2,201809,999912,201809,201902,999912,999912,96,1,0,0,1,5,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
cont.columns

Index(['SCTR_CD', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD', 'INS_CT_STAT_CD',
       'CT_CNCLS_DT', 'CT_TRMNT_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_EFCTV_DT',
       'CT_RST_DT', 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD', 'IS_CT_DIGNS',
       'CT_JDGMT_CD', 'CT_CHNL_CD', 'YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006'],
      dtype='object')

## (2) 단체보험 drop

In [62]:
cont = cont.loc[cont['IS_GRP_INS']==0]
cont.drop(columns =['IS_GRP_INS'], inplace=True)
cont.head()

,SCTR_CD,POL_SN,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## (3) 데이터프레임 정리

In [56]:
rlps = rlps[['JOIN_SN', 'POL_SN']]
rlps

,JOIN_SN,POL_SN
0,8,795034
1,8,3700186
2,8,6394418
3,8,6614027
4,8,7083655
...,...,...
196190,1959698,7255079
196191,1959698,7435599
196192,1959698,8764404
196193,1959919,2748732


In [72]:
cont

,SCTR_CD,POL_SN,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303236,8,8914008,11,2,201901,999912,201901,202901,999912,999912,360,2,5,0,1,4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303238,8,6242384,7,2,201212,999912,201212,203812,999912,999912,47,2,10,0,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303239,6,6050915,7,7,200902,201107,200902,201902,999912,999912,15,2,10,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303240,6,9901184,14,2,201809,999912,201809,201902,999912,999912,96,1,0,0,1,5,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
merged = pd.merge(rlps, cont, on='POL_SN', how='left')
merged = merged[['JOIN_SN', 'POL_SN', 'SCTR_CD',
                 'INS_CT_STAT_CD', 'CT_CNCLS_DT', 'CT_TRMNT_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_EFCTV_DT', 'CT_RST_DT',
                 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD']]
merged.head()

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
0,8,795034,3.0,2.0,201601.0,999912.0,201601.0,209001.0,999912.0,999912.0,94.0,2.0,20.0
1,8,795034,3.0,7.0,201601.0,201903.0,201601.0,209001.0,999912.0,999912.0,94.0,2.0,20.0
2,8,3700186,3.0,7.0,201110.0,201903.0,201110.0,206910.0,999912.0,999912.0,30.0,2.0,15.0
3,8,3700186,3.0,2.0,201110.0,999912.0,201110.0,206910.0,999912.0,999912.0,30.0,2.0,15.0
4,8,6394418,6.0,2.0,201110.0,999912.0,201110.0,208910.0,999912.0,999912.0,66.0,2.0,30.0


In [77]:
# merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300388 entries, 0 to 300387
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   JOIN_SN         228710 non-null  float64
 1   POL_SN          300388 non-null  int64  
 2   SCTR_CD         300388 non-null  int64  
 3   INS_CT_STAT_CD  300388 non-null  int64  
 4   CT_CNCLS_DT     300388 non-null  int64  
 5   CT_TRMNT_DT     300388 non-null  int64  
 6   CT_ST_DT        300388 non-null  int64  
 7   CT_END_DT       300388 non-null  int64  
 8   CT_EFCTV_DT     300388 non-null  int64  
 9   CT_RST_DT       300388 non-null  int64  
 10  CT_PY_AMT       300388 non-null  int64  
 11  CT_PY_CYCLE_CD  300388 non-null  int64  
 12  CT_PY_PD        300388 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 29.8 MB


In [58]:
# JOIN_SN NULL 삭제
merged = merged.dropna(subset=['JOIN_SN'])

# JOIN_SN int64로 변경
merged['JOIN_SN'] = merged['JOIN_SN'].astype('int64')
merged

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
0,8,795034,3.0,2.0,201601.0,999912.0,201601.0,209001.0,999912.0,999912.0,94.0,2.0,20.0
1,8,795034,3.0,7.0,201601.0,201903.0,201601.0,209001.0,999912.0,999912.0,94.0,2.0,20.0
2,8,3700186,3.0,7.0,201110.0,201903.0,201110.0,206910.0,999912.0,999912.0,30.0,2.0,15.0
3,8,3700186,3.0,2.0,201110.0,999912.0,201110.0,206910.0,999912.0,999912.0,30.0,2.0,15.0
4,8,6394418,6.0,2.0,201110.0,999912.0,201110.0,208910.0,999912.0,999912.0,66.0,2.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253683,1959698,7435599,6.0,7.0,201402.0,201905.0,201402.0,203402.0,999912.0,999912.0,24.0,2.0,20.0
253684,1959698,8764404,6.0,2.0,200803.0,999912.0,200803.0,202803.0,999912.0,999912.0,5.0,2.0,20.0
253685,1959919,2748732,6.0,5.0,200705.0,201005.0,200705.0,201005.0,999912.0,999912.0,1000.0,2.0,3.0
253686,1959919,3831658,3.0,1.0,201904.0,999912.0,201904.0,999912.0,999912.0,999912.0,80.0,2.0,5.0


In [59]:
merged_sort = merged.sort_values(by='JOIN_SN')
merged_sort

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
0,8,795034,3.0,2.0,201601.0,999912.0,201601.0,209001.0,999912.0,999912.0,94.0,2.0,20.0
14,8,9074314,3.0,1.0,201903.0,999912.0,201903.0,999912.0,999912.0,999912.0,140.0,2.0,20.0
13,8,8841475,6.0,2.0,200908.0,999912.0,200908.0,208908.0,999912.0,999912.0,91.0,2.0,15.0
12,8,8841475,6.0,2.0,200908.0,999912.0,200908.0,208908.0,999912.0,999912.0,75.0,2.0,15.0
11,8,8841475,6.0,2.0,200908.0,999912.0,200908.0,208908.0,999912.0,999912.0,88.0,2.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253672,1959698,3210920,8.0,2.0,201903.0,999912.0,201903.0,202903.0,999912.0,999912.0,5000.0,1.0,0.0
253684,1959698,8764404,6.0,2.0,200803.0,999912.0,200803.0,202803.0,999912.0,999912.0,5.0,2.0,20.0
253686,1959919,3831658,3.0,1.0,201904.0,999912.0,201904.0,999912.0,999912.0,999912.0,80.0,2.0,5.0
253685,1959919,2748732,6.0,5.0,200705.0,201005.0,200705.0,201005.0,999912.0,999912.0,1000.0,2.0,3.0


- 개인에 대한 전체계약건수 대비 해지건수 

- 증권번호로 개인에 대한 데이터는 다 있으니 / 증권번호의 갯수를 카운트 후 / 해지건수만 카운트하여 비율 구하기

In [60]:
merged_sort_grp = merged_sort.groupby('JOIN_SN')['POL_SN'].nunique().reset_index()
merged_sort_grp

,JOIN_SN,POL_SN
0,8,9
1,18,1
2,98,3
3,133,4
4,182,9
...,...,...
41771,1959548,4
41772,1959634,6
41773,1959666,4
41774,1959698,16


In [61]:
get357819 = merged_sort_grp[merged_sort_grp['JOIN_SN'] == 357819]
get357819

,JOIN_SN,POL_SN
7514,357819,25


# 1. 개인별 보험계약건수

In [62]:
join_sn_counts = merged_sort_grp[['JOIN_SN', 'POL_SN']]
join_sn_counts['계약건수'] = join_sn_counts['POL_SN']
join_sn_counts = join_sn_counts.drop(columns='POL_SN')
join_sn_counts

,JOIN_SN,계약건수
0,8,9
1,18,1
2,98,3
3,133,4
4,182,9
...,...,...
41771,1959548,4
41772,1959634,6
41773,1959666,4
41774,1959698,16


In [63]:
get357819_2 = join_sn_counts[join_sn_counts['JOIN_SN'] == 357819]
get357819_2

,JOIN_SN,계약건수
7514,357819,25


# 2. 해지건수

- 개인에 대한 전체계약건수 대비 해지건수 

- 증권번호로 개인에 대한 데이터는 다 있으니 / 증권번호의 갯수를 카운트 후 / 해지건수만 카운트하여 비율 구하기

In [64]:
ins_cd_7_counts = merged_sort[merged_sort['INS_CT_STAT_CD'] == 7]['JOIN_SN'].value_counts().reset_index()
ins_cd_7_counts.columns = ['JOIN_SN', '해지건수']
ins_cd_7_counts

,JOIN_SN,해지건수
0,748477,25
1,19979,23
2,1226045,23
3,1433558,20
4,1166691,20
...,...,...
20120,1272039,1
20121,1272061,1
20122,688917,1
20123,688649,1


# 3. 만기건수

In [65]:
ins_cd_5_counts = merged_sort[merged_sort['INS_CT_STAT_CD'] == 5]['JOIN_SN'].value_counts().reset_index()
ins_cd_5_counts.columns = ['JOIN_SN', '만기건수']
ins_cd_5_counts

,JOIN_SN,만기건수
0,1026251,26
1,1750242,23
2,1910039,23
3,1178681,23
4,195798,22
...,...,...
11925,814655,1
11926,814219,1
11927,813909,1
11928,813738,1


# 4. 데이터프레임 합치기

In [76]:
new = pd.merge(merged_sort_grp, ins_cd_7_counts, on='JOIN_SN', how='left')
new = pd.merge(new, ins_cd_5_counts, on='JOIN_SN', how='left')
new = new.fillna(0)
new

,JOIN_SN,POL_SN,해지건수,만기건수
0,8,9,4.0,0.0
1,18,1,0.0,1.0
2,98,3,0.0,1.0
3,133,4,1.0,0.0
4,182,9,2.0,0.0
...,...,...,...,...
41771,1959548,4,2.0,0.0
41772,1959634,6,0.0,0.0
41773,1959666,4,0.0,0.0
41774,1959698,16,7.0,0.0


In [77]:
get357819_2 = new[new['JOIN_SN'] == 357819]
get357819_2

,JOIN_SN,POL_SN,해지건수,만기건수
7514,357819,25,2.0,18.0


In [78]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41776 entries, 0 to 41775
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   JOIN_SN  41776 non-null  int64  
 1   POL_SN   41776 non-null  int64  
 2   해지건수     41776 non-null  float64
 3   만기건수     41776 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 1.3 MB


In [79]:
new['해지건수'] = new['해지건수'].astype(int)
new['만기건수'] = new['만기건수'].astype(int)

new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41776 entries, 0 to 41775
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   JOIN_SN  41776 non-null  int64
 1   POL_SN   41776 non-null  int64
 2   해지건수     41776 non-null  int32
 3   만기건수     41776 non-null  int32
dtypes: int32(2), int64(2)
memory usage: 979.3 KB


In [80]:
new['해지율'] = new['해지건수'] / new['POL_SN'] 
new['만기율'] = new['만기건수'] / new['POL_SN']
new

,JOIN_SN,POL_SN,해지건수,만기건수,해지율,만기율
0,8,9,4,0,0.444444,0.000000
1,18,1,0,1,0.000000,1.000000
2,98,3,0,1,0.000000,0.333333
3,133,4,1,0,0.250000,0.000000
4,182,9,2,0,0.222222,0.000000
...,...,...,...,...,...,...
41771,1959548,4,2,0,0.500000,0.000000
41772,1959634,6,0,0,0.000000,0.000000
41773,1959666,4,0,0,0.000000,0.000000
41774,1959698,16,7,0,0.437500,0.000000


In [81]:
new['해지율'] = new['해지율'].round(2)
new['만기율'] = new['만기율'].round(2)
new

,JOIN_SN,POL_SN,해지건수,만기건수,해지율,만기율
0,8,9,4,0,0.44,0.00
1,18,1,0,1,0.00,1.00
2,98,3,0,1,0.00,0.33
3,133,4,1,0,0.25,0.00
4,182,9,2,0,0.22,0.00
...,...,...,...,...,...,...
41771,1959548,4,2,0,0.50,0.00
41772,1959634,6,0,0,0.00,0.00
41773,1959666,4,0,0,0.00,0.00
41774,1959698,16,7,0,0.44,0.00


In [82]:
new['보험계약건수'] = new['POL_SN']
new

,JOIN_SN,POL_SN,해지건수,만기건수,해지율,만기율,보험계약건수
0,8,9,4,0,0.44,0.00,9
1,18,1,0,1,0.00,1.00,1
2,98,3,0,1,0.00,0.33,3
3,133,4,1,0,0.25,0.00,4
4,182,9,2,0,0.22,0.00,9
...,...,...,...,...,...,...,...
41771,1959548,4,2,0,0.50,0.00,4
41772,1959634,6,0,0,0.00,0.00,6
41773,1959666,4,0,0,0.00,0.00,4
41774,1959698,16,7,0,0.44,0.00,16


In [83]:
new = new[['JOIN_SN', '보험계약건수', '해지건수', '만기건수', '해지율', '만기율']]
new

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
0,8,9,4,0,0.44,0.00
1,18,1,0,1,0.00,1.00
2,98,3,0,1,0.00,0.33
3,133,4,1,0,0.25,0.00
4,182,9,2,0,0.22,0.00
...,...,...,...,...,...,...
41771,1959548,4,2,0,0.50,0.00
41772,1959634,6,0,0,0.00,0.00
41773,1959666,4,0,0,0.00,0.00
41774,1959698,16,7,0,0.44,0.00


In [84]:
get357819_2 = new[new['JOIN_SN'] == 357819]
get357819_2

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
7514,357819,25,2,18,0.08,0.72


In [85]:
new.to_csv('만기_해지_비율(fin).csv', index=False)

In [86]:
ratio = pd.read_csv('만기_해지_비율(fin).csv')
ratio

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
0,8,9,4,0,0.44,0.00
1,18,1,0,1,0.00,1.00
2,98,3,0,1,0.00,0.33
3,133,4,1,0,0.25,0.00
4,182,9,2,0,0.22,0.00
...,...,...,...,...,...,...
41771,1959548,4,2,0,0.50,0.00
41772,1959634,6,0,0,0.00,0.00
41773,1959666,4,0,0,0.00,0.00
41774,1959698,16,7,0,0.44,0.00
